In [ ]:
import pandas as pd; import numpy as np; import joblib; import warnings
warnings.filterwarnings('ignore')
DATA_FILE_PATH = "/kaggle/input/real-time-data/features.csv"
MODEL_FILE_PATH = "/kaggle/input/models/artifacts/xgboost_model.pkl"
TRANSFORMER_FILE_PATH = "/kaggle/input/models/artifacts/power_transformer.pkl"
OUTPUT_FILE_PATH = "/kaggle/working/prediction_output.csv"
print("--- Loading Files ---")
try:
    new_data_df = pd.read_csv(DATA_FILE_PATH)
    print(f"✅ Loaded data for address: {new_data_df['ADDRESS'].iloc[0]}")
except Exception as e: print(f"❌ Error loading data file: {e}"); exit()
try:
    model = joblib.load(MODEL_FILE_PATH)
    print("✅ Loaded XGBoost model.")
except Exception as e: print(f"❌ Error loading model file: {e}"); exit()
try:
    pt = joblib.load(TRANSFORMER_FILE_PATH)
    print("✅ Loaded PowerTransformer.")
except Exception as e: print(f"❌ Error loading transformer file: {e}"); exit()
print("\n--- Preprocessing New Data ---")
transformer_columns = pt.feature_names_in_
X_new_trans = pd.DataFrame(columns=transformer_columns)
for col in transformer_columns: X_new_trans[col] = new_data_df[col] if col in new_data_df.columns else 0
print("Applying Yeo-Johnson transformation...")
X_transformed = pt.transform(X_new_trans)
X_transformed_df = pd.DataFrame(X_transformed, columns=transformer_columns)
training_columns = model.feature_names_in_
X_final = pd.DataFrame(columns=training_columns)
for col in training_columns: X_final[col] = X_transformed_df[col] if col in X_transformed_df.columns else 0
print("✅ Data transformed and aligned.")
print("\n--- Making Prediction ---")
prediction = model.predict(X_final)
predicted_label = int(prediction[0])
prediction_proba = model.predict_proba(X_final)
confidence_score = float(prediction_proba[0].max())
print(f"Predicted Label: {predicted_label}")
print(f"Confidence Score: {confidence_score:.2%}")
print("\n--- Saving Output ---")
result_dict = {'address': new_data_df['ADDRESS'].iloc[0], 'prediction': predicted_label, 'confidence': confidence_score}
result_df = pd.DataFrame([result_dict])
result_df.to_csv(OUTPUT_FILE_PATH, index=False)
print(f"✅ Prediction saved to {OUTPUT_FILE_PATH}")